In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
data_location = ''

In [3]:
# Read in the data set
file_name = 'competition_starter.csv'
starter_data = pd.read_csv(data_location + file_name, header=0, index_col=0)

In [4]:
# First, read in the csv
file_name_employment = 'competition_employment.csv'
employment_data = pd.read_csv(data_location + file_name_employment, header=0)

In [5]:
# Perform the merge
common_key = 'soa'

hack_data = starter_data.merge(employment_data, on=common_key, how='left')

In [6]:
# First, read in the csv
file_name_waiting = 'competition_waiting.csv'
waiting_data = pd.read_csv(data_location + file_name_waiting)

In [7]:
waiting_data['gp_month'] = [int(i.split('-')[0]) for i in waiting_data['quarter_end']]
waiting_data['gp_year'] = [int(i.split('-')[1]) for i in waiting_data['quarter_end']]

In [8]:
waiting_data = waiting_data.drop(['quarter_end'], 1)

In [9]:
# Perform the merge
common_key = ['hsc_trust', 'gp_month', 'gp_year']

hack_data = hack_data.merge(waiting_data, on=common_key, how='left')

In [10]:
y = hack_data["gp_cost_per_registered_patient"].values
hack_data = hack_data.drop("gp_cost_per_registered_patient", axis = 1)
hack_data.shape

(9293, 192)

In [11]:
hack_data.isnull().sum()

gp_practice                       0
gp_year                           0
gp_month                          0
quality                           0
eastings                          0
northings                         0
country                           0
nhs_ha                            0
longitude                         0
latitude                          0
parliamentary_constituency        0
european_electoral_region         0
primary_care_trust                0
incode                            0
outcode                           0
admin_district                    0
admin_ward                        0
ccg                               0
nuts                              0
multiple_dep_measure_rank      1101
income_score                   1101
income_rank                    1101
empl_score_18-59_64_yrs        1101
empl_rank_18-59_64_yrs         1101
health_dep_disability_rank     1101
education_skills_train_rank    1101
proximity_to_services_rank     1101
living_env_rank             

In [12]:
holdout = pd.read_csv(data_location + "holdout.csv", header=0, index_col=0)

In [13]:
#check to see if holdout set shares the same col names as the dataset, so not all are shared by the results
shared_col_names = np.intersect1d(np.asarray(list(holdout)), np.asarray(list(hack_data)))
holdout_list = list(holdout)
for col_name in shared_col_names:
    if col_name in holdout_list:
        holdout_list.remove(col_name)
holdout_list

['record_id',
 'All_usual_residents',
 'Very_good_health',
 'Good_health',
 'Fair_health',
 'Bad_health',
 'Very_bad_health',
 'quarter_end']

In [14]:
#Define numeric values and categorical values

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_values = list(hack_data.select_dtypes(include=numerics))

#Get numeric column variables that act as categorical values

categoric_values = list(hack_data)
for value in numeric_values:
    categoric_values.remove(value)
    
        
print(len(numeric_values))
print(len(categoric_values))

179
13


In [15]:
print(len(numeric_values))
print(len(categoric_values))

179
13


In [16]:
hack_data[numeric_values].isnull().sum()

gp_practice                                   0
gp_year                                       0
gp_month                                      0
quality                                       0
eastings                                      0
northings                                     0
longitude                                     0
latitude                                      0
multiple_dep_measure_rank                  1101
income_score                               1101
income_rank                                1101
empl_score_18-59_64_yrs                    1101
empl_rank_18-59_64_yrs                     1101
health_dep_disability_rank                 1101
education_skills_train_rank                1101
proximity_to_services_rank                 1101
living_env_rank                            1101
crime_disorder_rank                        1101
income_dep_children_score                  1101
income_dep_children_rank                   1101
income_dep_older_people_score           

In [17]:
#Check what values are in soa categorical column
np.unique(str(hack_data['soa']))

array([ '0         Island 2\n1         Island 2\n2         Island 2\n3         Island 2\n4         Island 2\n5         Island 2\n6         Island 2\n7         Island 2\n8         Island 2\n9         Island 2\n10        Island 2\n11        Island 2\n12        Island 2\n13        Island 2\n14        Island 2\n15        Island 2\n16        Island 2\n17        Island 2\n18        Island 2\n19        Island 2\n20        Island 2\n21        Island 2\n22        Island 2\n23        Island 2\n24        Island 2\n25        Island 2\n26        Island 2\n27        Island 2\n28        Island 2\n29        Island 2\n           ...    \n9263    Plumbridge\n9264    Plumbridge\n9265    Plumbridge\n9266    Plumbridge\n9267    Plumbridge\n9268    Plumbridge\n9269    Plumbridge\n9270    Plumbridge\n9271    Plumbridge\n9272    Plumbridge\n9273    Plumbridge\n9274    Plumbridge\n9275    Plumbridge\n9276    Plumbridge\n9277    Plumbridge\n9278    Plumbridge\n9279    Plumbridge\n9280    Plumbridge\n9281    Plu

In [18]:
#As we can see we have to replace Nan
hack_data['soa'][hack_data['soa'].isnull()].head()

410    NaN
411    NaN
412    NaN
413    NaN
414    NaN
Name: soa, dtype: object

In [19]:
#Build Feature Pipeline which performs several functions,
#
#   -Selects the approapriate dataframe according to whether its categorical or numeric
#
#   -Tries to fill missing numeric data using the median
#
#   -One Hot Encodes the categorical value for each categorical column
#
#   -Standard Scales the numerical data
#

from sklearn.preprocessing import StandardScaler, Imputer
from utils.DataFrameSelector import DataFrameSelector

hack_data['soa'].replace(np.nan, 'unknown')

imputer = Imputer(strategy = 'median')

In [20]:
#Only numeric pipe here and concatenate on categoric columns
imputed_hack = imputer.fit_transform(hack_data[numeric_values])
processed_hack = np.concatenate((imputed_hack, hack_data[categoric_values].values), axis = 1)

In [21]:
pd.DataFrame(processed_hack, columns = list(hack_data)).describe()

,gp_practice,gp_year,gp_month,quality,eastings,northings,country,nhs_ha,longitude,latitude,...,palliative_medicine,plastic_surgery,radiology,rehabilitation,restorative_dentistry,rheumatology,t_o_surgery,thoracic_medicine,thoracic_surgery,urology
count,9293.0,9293.0,9293.0,9293.0,9293.0,9293.0,9293.000000,9293.000000,9293.0,9293.00,...,9293,9293,9293,9293,9293,9293,9293,9293,8192,9293
unique,227.0,4.0,12.0,2.0,179.0,179.0,179.000000,179.000000,141.0,48.00,...,1,5,173,67,11,143,5,5,141,5
top,900870.0,2015.0,3.0,1.0,336515.0,353478.0,-6.042344,54.510298,348.0,0.27,...,Northern Ireland,Northern,1NS,BT48,Belfast,Killycomain,Northern,East of Northern Ireland,Hilden 1,Northern
freq,41.0,2724.0,907.0,9170.0,246.0,246.0,246.000000,246.000000,1142.0,1634.00,...,9293,2337,246,369,1845,246,2337,2460,246,2337


In [22]:
processed_hack_df = pd.DataFrame(processed_hack, columns = list(hack_data))
processed_hack_df["gp_cost_per_registered_patient"] = y

In [23]:
thoracic_surgery_list = processed_hack_df['thoracic_surgery'].values

In [24]:
for i, surgery in enumerate(thoracic_surgery_list):
    if surgery is np.nan:
        thoracic_surgery_list[i] = 'unknown'
        
processed_hack_df['thoracic_surgery'] = thoracic_surgery_list

In [25]:
processed_hack_df.isnull().sum().sum()

0

In [26]:
processed_hack_df.head()

,gp_practice,gp_year,gp_month,quality,eastings,northings,country,nhs_ha,longitude,latitude,...,plastic_surgery,radiology,rehabilitation,restorative_dentistry,rheumatology,t_o_surgery,thoracic_medicine,thoracic_surgery,urology,gp_cost_per_registered_patient
0,900207,2014,1,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,15.290104
1,900207,2014,2,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,11.646997
2,900207,2014,3,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,13.544038
3,900207,2014,4,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,13.724001
4,900207,2014,5,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,14.169956


In [27]:
processed_hack_df['record_id'] = list(range(processed_hack_df.shape[0]))

In [28]:
# linux_path = '/home/mark/Documents/Kaggle_Projects/Allstate-Data-Hack-2017/input/train.csv'
# windows_path = 'C:/Users/srder/Documents/GitHub/ALlstate-Data-Hack-2017/input/datasets/train.csv'
# processed_hack_df.to_csv(windows_path)

In [29]:
file_name = 'holdout.csv'
holdout_data = pd.read_csv(data_location + file_name, header=0, index_col=0)

In [30]:
holdout_data = holdout_data.drop(holdout_list, axis = 1)
holdout_data['soa'] = holdout_data['soa'].replace(np.nan, 'unknown')
holdout_data['hsc_trust'] = holdout_data['hsc_trust'].replace(np.nan, 'unknown')

In [31]:
numeric_values = list(holdout_data.select_dtypes(include=numerics))

#Get numeric column variables that act as categorical values

categoric_values = list(holdout_data)
for value in numeric_values:
    categoric_values.remove(value)
    
imputed_holdout_num = imputer.fit_transform(holdout_data[numeric_values])
processed_holdout = np.concatenate((imputed_holdout_num, holdout_data[categoric_values].values), axis = 1)

In [32]:
processed_holdout_df = pd.DataFrame(processed_holdout, columns = list(numeric_values + categoric_values))

In [33]:
processed_hack_df.to_csv('/home/mark/Documents/Kaggle_Projects/Allstate-Data-Hack-2017/input/train.csv')

In [34]:
processed_holdout_df['record_id'] = list(range(processed_holdout_df.shape[0]))

In [35]:
processed_holdout_df.to_csv('/home/mark/Documents/Kaggle_Projects/Allstate-Data-Hack-2017/input/test.csv')

In [36]:
processed_holdout_df

,gp_practice,gp_year,gp_month,quality,eastings,northings,longitude,latitude,multiple_dep_measure_rank,income_score,...,primary_care_trust,incode,outcode,admin_district,admin_ward,ccg,nuts,soa,hsc_trust,record_id
0,900207,2017,6,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,0
1,900207,2017,7,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,unknown,1
2,900207,2017,8,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,unknown,2
3,900209,2017,6,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,Belfast,3
4,900209,2017,7,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,unknown,4
5,900209,2017,8,1,336515,374326,-5.88846,54.5987,274,0.3,...,Belfast,1NS,BT4,Belfast,Connswater,Belfast,Belfast,Island 2,unknown,5
6,900211,2017,6,1,335833,373556,-5.89936,54.592,51,0.55,...,Belfast,5FB,BT5,Belfast,Beersbridge,Belfast,Belfast,The Mount 1,Belfast,6
7,900211,2017,7,1,335833,373556,-5.89936,54.592,51,0.55,...,Belfast,5FB,BT5,Belfast,Beersbridge,Belfast,Belfast,The Mount 1,unknown,7
8,900211,2017,8,1,335833,373556,-5.89936,54.592,51,0.55,...,Belfast,5FB,BT5,Belfast,Beersbridge,Belfast,Belfast,The Mount 1,unknown,8
9,900212,2017,6,1,336543,364858,-5.89241,54.5137,834,0.1,...,Belfast,8HR,BT8,Lisburn and Castlereagh,Carryduff West,Belfast,Outer Belfast,Carryduff West 2,Belfast,9
